In [11]:
import mdtraj as md
import mdfeature.features as features
import numpy as np
import dill
from mdfeature.ConvergenceAnalyser import free_energy_estimate_2D
import pyemma

In [12]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 2
font = {'family' : 'sans-serif',
        'size'   : 14.0}
mpl.rc('font', **font) 
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] =  16
mpl.rcParams['font.size'] =  15
mpl.rcParams['figure.autolayout'] =  True
mpl.rcParams['figure.figsize'] =  7.2,4.45
mpl.rcParams['axes.titlesize'] =  16
mpl.rcParams['axes.labelsize'] =  17
mpl.rcParams['lines.linewidth'] =  2
mpl.rcParams['lines.markersize'] =  6
mpl.rcParams['legend.fontsize'] =  13

Define Experiment Class

In [1]:
class Experiment:

        def __init__(self, pdb, trajectory, dihedral_pickle_file, temperature, length, record_interval, step_size):
                self.pdb, self.topology, self.trajectory, self.dihedral_traj = \
                        self.load_experiment(pdb, trajectory, dihedral_pickle_file)
                self.temperature = temperature
                self.length = length
                self.record_interval = record_interval
                self.step_size = step_size
                self.beta = 1/(self.temperature*0.0083144621)
                self.conformations = len(self.trajectory)

                self.PCA = None
                self.TICA = None
                self.VAMP = None

        @staticmethod
        def load_experiment(pdb, trajectory, dihedral_pickle_file):
                pdb_file = md.load_pdb(pdb)
                topology_file = pdb_file.topology
                original_trajectory = md.load_dcd(trajectory, topology_file)
                adjusted_trajectory = original_trajectory.superpose(original_trajectory[0])
                dihedral_traj = dill.load(open(dihedral_pickle_file, "rb"))

                return pdb_file, topology_file, adjusted_trajectory, dihedral_traj

        def ramachandran_plot(self, bins=100):
                # TODO: fix this function
                free_energy, xedges, yedges = free_energy_estimate_2D(self.dihedral_traj.T, self.beta, bins=bins)
                fig, ax = plt.subplots()
                im = ax.pcolormesh(xedges, yedges, free_energy.T)
                plt.colorbar(im)
                plt.xlabel(r'$\phi$')
                plt.ylabel(r'$\psi$')
                plt.gca().set_aspect('equal')
                plt.show()

        def compute_PCA(self, dim, stride=1):
                self.PCA = pyemma.coordinates.pca(self.trajectory, dim=dim, stride=stride)

        def compute_TICA(self, dim, lag, stride=1):
                self.TICA = pyemma.coordinates.tica(self.trajectory, lag=lag, dim=dim, stride=stride)

        def compute_VAMP(self, dim, lag, stride=1):
                self.VAMP = pyemma.coordinates.vamp(self.trajectory, lag=lag, dim=dim, stride=stride)


Tasks to do

1) Adapt convergence code to make it run with two empirical grids
2) Write code for learning collective variables from an experiment (PCA, TICA, VAMPnet) Done.
3) OpenMM metadynamics - in what form do the collective variables need to be given?
4) Learn collective variables from 20ns trajectory (yorkshire)
5) Diffusion maps vs. TICA vs. VAMPnet trajectories Kramers Rate in ring double well
6) Diffusion maps vs. TICA vs. VAMPnet trajectories Kramers Rate in alanine dipeptide